In [54]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
import glob
import re
import copy

In [55]:
shadow = 'shadow10_2'
evaluation_path = "../../get_data/evaluation_relay_nodes3/" + shadow + "/"
lsgo_file_list = []
# lsgo_files = glob.glob("../../ns3.30/data/lsgo/*")
lsgo_files = glob.glob(evaluation_path + "lsgo/*")
for file in lsgo_files:
    lsgo_file_list.append(file)
    
send_lsgo_file_list = []
# send_lsgo_files = glob.glob("../../ns3.30/data/send_lsgo/*")
send_lsgo_files = glob.glob(evaluation_path + "send_lsgo/*")
for file in send_lsgo_files:
    send_lsgo_file_list.append(file)
print('receive log')
print(lsgo_file_list)
print('send log')
print(send_lsgo_file_list)

receive log
['../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10001nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10002nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10003nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10004nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10005nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10006nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10007nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10008nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10009nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10010nodenum_400.csv', '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10011nodenum_400.csv

In [56]:
seed_index = 3
node_index = 4
seed_list = []
nodenum_list = ['400'] ###############nodenumの形式を変更したらここはベタ打ちで変更を加える
regex = re.compile('\d+')

for path in lsgo_file_list:
    for line in path.splitlines():
        match = regex.findall(line)
        if match[seed_index] not in seed_list:
            seed_list.append(match[seed_index])
            
print(seed_list)
print('simulation 試行回数',len(seed_list))

['10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '10010', '10011', '10012', '10013', '10014', '10015', '10016', '10017', '10018', '10019', '10020', '10021', '10022', '10023', '10024', '10025', '10026', '10027', '10028', '10029', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10037', '10038', '10039', '10040', '10041', '10042', '10043', '10044', '10045', '10046', '10047', '10048', '10049']
simulation 試行回数 49


In [57]:
lsgo_dict = {}
check_seed = seed_list[0]
lsgo_dict[check_seed] = {}
for path in lsgo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        lsgo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        lsgo_dict[match[seed_index]] = {}
        lsgo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新
        
send_lsgo_dict = {}
check_seed = seed_list[0]
send_lsgo_dict[check_seed] = {}
for path in send_lsgo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        send_lsgo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        send_lsgo_dict[match[seed_index]] = {}
        send_lsgo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新
        
print(lsgo_dict)
print(send_lsgo_dict)

{'10001': {'400': '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10001nodenum_400.csv'}, '10002': {'400': '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10002nodenum_400.csv'}, '10003': {'400': '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10003nodenum_400.csv'}, '10004': {'400': '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10004nodenum_400.csv'}, '10005': {'400': '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10005nodenum_400.csv'}, '10006': {'400': '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10006nodenum_400.csv'}, '10007': {'400': '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10007nodenum_400.csv'}, '10008': {'400': '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10008nodenum_400.csv'}, '10009': {'400': '../../get_data/evaluation_relay_nodes3/shadow10_2/lsgo\\lsgo-seed_10009nodenum_400.csv'}, '10010': {'400': '../../get

In [58]:
def packet_ratio(path):
    #csvファイルの取得
    path_df = pd.read_csv(path)
    dest_list = path_df['destination_id'].value_counts().index.tolist() #重複なしのdestination_idを格納
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'].value_counts().index.tolist()
#     PDR = len(reach_list)/len(dest_list)
    PDR = len(reach_list)/10
    
#     *************** len(dest_list): destination_idのリストをいったん10個で定義
    return PDR

In [59]:
def packet_hop(path):
    path_df = pd.read_csv(path)
    dest_list = path_df['destination_id'].value_counts().index.tolist() #重複なしのdestination_idを格納
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'] 
    #reach_listを回す
    total_hop = 0
    for reach_id in reach_list:
        hop = path_df[(path_df['destination_id'] == reach_id) & (path_df['destination_id'] == path_df['recv_id'])]['hopcount']
        hop = int(hop.head(1).iloc[-1])
        total_hop += hop
    if(len(reach_list) != 0):
        average_hop = total_hop / len(reach_list)
    else:
        average_hop = total_hop
    
    return average_hop

In [60]:
def packet_delay(path):
    delay_list = []
    #csvファイルの取得
    path_df = pd.read_csv(path)
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'].value_counts().index.tolist()
    #reach_listを回す
    if (len(reach_list) == 0):
        delay = 0
        print('reach_list none')
        return delay
    for reach_id in reach_list:
        source_time = path_df[path_df['destination_id'] == reach_id]['time']
        recv_time = path_df[(path_df['destination_id'] == reach_id) & (path_df['destination_id'] == path_df['recv_id'])]['time']
        delay = int(recv_time.head(1).iloc[-1]) - int(source_time.head(1).iloc[-1])
        delay = delay / 1000000
        delay_list.append(delay)
    delay_mean = np.mean(delay_list)
    return delay_mean

In [61]:
def packet_over_head_send(send_path, recv_path):
    send_path_df = pd.read_csv(send_path)
    send_column_num = len(send_path_df.columns) #カラム数を取得
#     display(path_df.iloc[:,send_column_num-1]) 末尾の取得方法　path_df = pd.read_csv(recv_path)末尾=送信ログ(送信したかどうか)
    #目的地に届いたidだけをlistに保存
    recv_path_df = pd.read_csv(recv_path)
    reach_list = recv_path_df[recv_path_df['recv_id'] == recv_path_df['destination_id']]['destination_id'].value_counts().index.tolist()
    send_count = send_path_df[send_path_df.iloc[:,send_column_num-1] == 1].value_counts() #最後のカラムの値が1
#     print(len(send_count))
    if(len(reach_list) != 0):
        overhead = len(send_count)/len(reach_list)
    else:
        overhead = len(send_count)
    return overhead

In [62]:
def col_average(seeds):
    new_list = []
    for seed in seeds:
        del seed[len(nodenum_list)] #1列ずつ平均を算出するうえで邪魔なseed値を消していく seedの個数 
        new_list.append(seed)
#     np.set_printoptions(precision=3, suppress=True)
    np_new_list = np.array(new_list)
    return np.mean(np_new_list, axis=0)

In [63]:
def header(protocol, writer, index):
    writer.writerow([])
    writer.writerow([protocol, index])
    writer.writerow(['200','300','400','seed'])

In [64]:
def write(seeds):
    for seed in seeds:
        writer.writerow(seed)

with open(evaluation_path + '/12_20.csv', 'w', newline='') as csv_file:
    #---------------------------PDR---------------------------
    lsgo_pdr_average_list = []
    lsgo_delay_average_list = []
    lsgo_overhead_average_list = []
    lsgo_hop_average_list = []
    
    #headerを設定
    writer = csv.writer(csv_file)
    #--lsgo---#
    header('LSGO', writer, 'PDR')
    for seed in seed_list:
        lsgo_pdr_ratio_list = []
        for node in nodenum_list:
            path = lsgo_dict[seed][node]
            ratio = packet_ratio(path)
            lsgo_pdr_ratio_list.append(ratio)
        lsgo_pdr_ratio_list.append(seed)
        writer.writerow(lsgo_pdr_ratio_list)
        lsgo_pdr_average_list.append(lsgo_pdr_ratio_list)
    #------------------------delay-----------------------
    #--lsgo--#
    header('LSGO', writer, 'delay')
    for seed in seed_list:
        lsgo_delay_list = []
        for node in nodenum_list:
            path = lsgo_dict[seed][node]
            delay = packet_delay(path)
            lsgo_delay_list.append(delay)
        lsgo_delay_list.append(seed)
        writer.writerow(lsgo_delay_list)

        lsgo_delay_average_list.append(lsgo_delay_list)
#         print('lsgo_delay_average_list', lsgo_delay_average_list)       
    #------------------------overhead-----------------------
    #--lsgo--#
    header('LSGO', writer, 'Overhead')
    for seed in seed_list:
        lsgo_overhead_list = []
        for node in nodenum_list:
            recv_path = lsgo_dict[seed][node]
            send_path = send_lsgo_dict[seed][node]
            overhead = packet_over_head_send(send_path, recv_path)
            lsgo_overhead_list.append(overhead)
        lsgo_overhead_list.append(seed)
        writer.writerow(lsgo_overhead_list)
        lsgo_overhead_average_list.append(lsgo_overhead_list)
        
    #------------------------Hop-----------------------
    header('LSGO', writer, 'Hop')
    for seed in seed_list:
        lsgo_hop_list = []
        for node in nodenum_list:
            path = lsgo_dict[seed][node]
            hop = packet_hop(path)
            lsgo_hop_list.append(hop)
        lsgo_hop_list.append(seed)
        writer.writerow(lsgo_hop_list)
        lsgo_hop_average_list.append(lsgo_hop_list)
        
    header('LSGO', writer , 'PDR-average')
    writer.writerow(col_average(lsgo_pdr_average_list))
    header('LSGO', writer , 'delay-average')
    writer.writerow(col_average(lsgo_delay_average_list))
    header('LSGO', writer , 'overhead-average')
    writer.writerow(col_average(lsgo_overhead_average_list))
    header('LSGO', writer , 'hop-average')
    writer.writerow(col_average(lsgo_hop_average_list))
    print('finish')

finish


In [65]:
print('lsgo-----------------')
for seed in seed_list:
        lsgo_delay_list = []
        for node in nodenum_list:
            if(node not in lsgo_dict[seed]):
                print("seed", seed, 'node_num', node)

lsgo-----------------
